In [8]:
import pandas as pd
import re
from rapidfuzz import fuzz, process

In [3]:
#Modulo que  agrega el path de los modulos al sys.path
from pathlib import Path

# Obtener la ruta al directorio "modulos"
ruta_modulos = Path("../modulos").resolve()

# Agregar la ruta al sys.path
if str(ruta_modulos) not in sys.path:
    sys.path.append(str(ruta_modulos))          

# Obtener la ruta al directorio "modulos"
ruta_modulos = Path("../modulos").resolve()
print(f"Ruta de modulos: {ruta_modulos}")  # Verifica la ruta que se obtiene


Ruta de modulos: C:\Users\OPERADOR\Desktop\PROYECTO DIGITALIZACION\digitalizacion_distriapp\backend\src\modulos


In [9]:
#MODULE DATA LOADER 

# Carga de datos
def load_data(df_archivo):
    """
    Carga un archivo CSV en un DataFrame de pandas.

    Parámetros:
        archivo (str): El nombre o la ruta del archivo CSV.

    Retorna:
        DataFrame: El DataFrame que contiene los datos del archivo CSV.
    """
    try:
        # Cargar el archivo CSV en un DataFrame
        df = pd.read_csv(df_archivo)
        return df
    except Exception as e:
        print(f"Error al cargar el archivo {df_archivo}: {e}")
        return None

# Variables del DF
name_df_provedor = "prueba2.csv" 
name_df_productos =  "L_STMaestroPrd.csv"     

#Variables Funcion  lodad_Data
df_proveedor = load_data(name_df_provedor)
df_productos = load_data(name_df_productos)

#Copias del DataFrame 
copy_dfProductos = df_productos.loc[:, df_productos.notna().any()].copy()
copy_dfProveedores = df_proveedor

C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_14060\613300319.py:16: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_archivo)


In [5]:
# Module: SEARCH UTILS : Enfoque busqueda de laboratorio

def laboratorio_input_df(df, columna):
    """
    Busca coincidencias parciales en una columna de un DataFrame basado en una entrada del usuario.

    Parámetros:
        df (DataFrame): El DataFrame en el que se buscarán las coincidencias.
        columna (str): El nombre de la columna en la que se realizará la búsqueda.

    Solicita:
        str: Nombre del laboratorio ingresado por el usuario.

    Retorna:
        DataFrame: Un DataFrame con las coincidencias encontradas.
        None: Si no se encuentran coincidencias.
|
    Ejemplo de uso:
        nombre_columna = 'atrib0'
        laboratorio_coincidencias = laboratorio_input_df(copy_dfProductos, nombre_columna)
    """
    # Solicitar el nombre del laboratorio al usuario
    input_name = input("Ingrese el nombre del laboratorio: ").strip()

    # Buscar coincidencias parciales en la columna especificada
    tablas_coincidencias = df[df[columna].str.contains(input_name, case=False, na=False)]

    # Verificar y retornar los resultados
    if not tablas_coincidencias.empty:
        print("Se encontraron valores coincidentes:")
        return tablas_coincidencias
    else:
        print("No se encontraron coincidencias.")
        return None


#Siempre se le manda la copia del DF
columna_nombre = 'atrib0'

# Búsqueda de coincidencias en el DataFrame

# Buscar coincidencias en el DataFrame de productos según el nombre del laboratorio proporcionado
laboratorio_coincidencias_df = laboratorio_input_df(
    copy_dfProductos,  # Copia del DataFrame de productos
    columna_nombre  # Nombre de la columna a buscar
)

# Imprimir DF DEL LABORATORIO COINCIDENTE
# Se debe usar este data frame para hacer las coincidencias con la lista de proveedores
# que envia el proveedor.  
laboratorio_coincidencias_df  



Se encontraron valores coincidentes:


,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,ninum,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2
0,122703.0,MEE-EST-00904,12185,-,-,-,EM,NaN,NaN,UN,...,NaN,E,0.0,NaN,ITGEV,0,0.0,JAYOR,NaN,NaN
3,125531.0,DEC-PGM-03064,48457,BOLSAS PAPEL GRADO MEDICO 175X65X420MM - KIMS,BOLSA X 500 UNIDADES UM: BOLSA,PAPEL GRADOMEDICO 17,MP,NaN,NaN,UN,...,NaN,E,0.0,NaN,ITG,0,0.0,ETILEN,NaN,NaN
4,133202.0,DEC-PGM-03110,EPBL-19,BOLSAS PAPEL GRADO MEDICO 260X60X520MM - KIMS,PACK X 1000 UNIDADES UM: PACK,16/08/24,MP,NaN,NaN,UN,...,NaN,E,0.0,NaN,ITG,0,0.0,ETILEN,NaN,NaN
5,133201.0,DEC-PGM-03109,EPBL-23,BOLSAS PAPEL GRADO MEDICO 320X120X600MM - KIMS,PACK X 1000 UNIDADES UM: PACK,ult.actual.28/10/24,MP,NaN,NaN,UN,...,NaN,E,0.0,NaN,ITG,0,0.0,ETILEN,NaN,NaN
6,125926.0,DEC-PGM-03065,BOL,BOLSAS PAPEL GRADO MEDICO 330X100X625 MM (X500...,BOLSA X 500 UNIDADES UM: BOLSA,BOLSA,MP,NaN,NaN,UN,...,NaN,E,0.0,NaN,ITG,0,0.0,ETILEN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56769,118010.0,49702,49702,ZYVALIX 250 mg comp.x 120,NaN,ZYVALIX 250 mg comp.,PT,7795367001885.0,LKM_B,UN,...,NaN,E,0.0,NaN,ITGEV,0,0.0,LKM ONCO/ESPECI,NaN,NaN
56770,129759.0,61433,61433,ZYVALIX PLUS 500 mg comp.x 60,NaN,ZYVALIX PLUS 500 mg,PT,7795367552196.0,NaN,UN,...,NaN,E,0.0,NaN,ITGEV,0,0.0,LKM ONCO/ESPECI,NaN,NaN
56771,26456.0,27450,27450,ZYVOX IV 600 mg bolsas x 10x300ml,CAJA X 10 BOLSAS X 300ML - UM: CAJA,ZYVOX IV 600 mg bols,PT,7791824117007.0,BOEHRINGER,UN,...,101.0,E,0.0,NaN,ITGEV,0,0.0,PFIZER,NaN,NaN
56772,26457.0,27451,27451,ZYVOX ORAL 600 mg tab.x 10,NaN,ZYVOX ORAL 600 mg ta,PT,7791824116994.0,NaN,UN,...,101.0,E,0.0,NaN,ITGEV,0,0.0,PFIZER,NaN,NaN


In [6]:
#MODULO data_normalization: Enfoque de estandarización de datos.

#Funcion REGEX
def clear_column(column):
    """
    Limpia y normaliza el texto en una columna para eliminar términos irrelevantes y uniformar el formato.

    Esta función utiliza expresiones regulares para eliminar números, unidades de medida, abreviaturas comunes, 
    caracteres especiales y palabras redundantes. También convierte el texto a minúsculas, elimina espacios en 
    blanco adicionales y normaliza el formato para facilitar su análisis posterior.

    Parámetros:
        column (str): El texto de la columna a limpiar.

    Retorna:
        str: El texto limpio y normalizado.

    Pasos de limpieza:
        1. Elimina números, unidades de medida y términos irrelevantes (e.g., "mg", "blister", "ml").
        2. Elimina caracteres especiales como dos puntos, puntos, comas, guiones, etc.
        3. Elimina espacios en blanco múltiples y normaliza el texto.
        4. Convierte el texto a minúsculas para uniformidad.
        5. Elimina palabras vacías comunes (e.g., "de", "la", "el").
        6. Elimina palabras repetidas consecutivas (e.g., "producto producto").
        7. Elimina términos relacionados con la dosis o presentación (e.g., "20mg", "30ml").

    Ejemplo de uso:
        texto_limpio = clear_column("20mg Blister de 10 unidades: Marca X")
        print(texto_limpio)  # Output: "marca x"
    """
    # Expresión regular para eliminar números, unidades, abreviaturas y términos irrelevantes
    columna_limpia = re.sub(
        r'\b(\d+|mg|comp\.?|blisters?|estuche|x|ml|unidades|marca|botella|caja|referencia|cm|g|m|f\.a\.x|'
        r'jbe\.|sol\.|env\.|lata|vial|pvo\.|sobres|gotero|cápsulas|frascos?|blister|bot\.|mcg|sp|p\.b\.)\b', 
        '', 
        column, 
        flags=re.IGNORECASE
    )
    
    # Eliminar caracteres especiales como los dos puntos, puntos finales, guiones, comas, etc.
    columna_limpia = re.sub(r'[:\.\-,;()]', '', columna_limpia)

    # Eliminar múltiples espacios en blanco y normalizar el texto
    columna_limpia = re.sub(r'\s+', ' ', columna_limpia).strip()

    # Convertir todo el texto a minúsculas
    columna_limpia = columna_limpia.lower()

    # Eliminar palabras vacías comunes
    columna_limpia = re.sub(r'\b(?:de|la|el|en|a|y|con|para|del|sobre|por|medicamento|tratamiento|ac)\b', '', columna_limpia).strip()

    # Eliminar palabras repetidas consecutivas
    columna_limpia = re.sub(r'\b(\w+)\s+\1\b', r'\1', columna_limpia)

    # Eliminar términos relacionados con dosis o presentación
    columna_limpia = re.sub(r'\b(\d+mg|\d+g|\d+ml)\b', '', columna_limpia)

    return columna_limpia

In [7]:
#MODULE DataFrame_Column_Matcher: Busca coincidencias entre las dos tablas

# Función para obtener coincidencias entre las columnas de dos DataFrames y agregar una columna 'match'
def obtener_coincidencias_con_match(productos_df, proveedores_df, col_productos, col_proveedores):
    """
    Busca coincidencias entre las columnas especificadas de dos DataFrames y agrega una columna 'match' que 
    indica si hubo una coincidencia o no. Si se encuentra una coincidencia, combina los datos relevantes 
    de ambos DataFrames.

    Parámetros:
        productos_df (DataFrame): DataFrame que contiene la información de los productos.
        proveedores_df (DataFrame): DataFrame que contiene la información de los proveedores.
        col_productos (str): Nombre de la columna en el DataFrame de productos que será comparada.
        col_proveedores (str): Nombre de la columna en el DataFrame de proveedores que será comparada.

    Retorna:
        DataFrame: Un DataFrame resultante que incluye:
            - Columnas combinadas de ambos DataFrames en caso de coincidencia.
            - Una columna 'match' que indica True si hay coincidencia, False en caso contrario.

    Pasos del proceso:
        1. Se limpian las columnas relevantes de ambos DataFrames usando la función `clear_column`.
        2. Se iteran las filas del DataFrame de proveedores y se buscan coincidencias en el DataFrame de productos.
        3. Si hay coincidencias, se combinan las filas correspondientes de ambos DataFrames.
        4. Si no hay coincidencias, se agrega la fila del proveedor con valores nulos para las columnas de productos.
    """
    # Crear copias explícitas de los DataFrames para evitar problemas con modificaciones directas
    productos_df = productos_df.copy()
    proveedores_df = proveedores_df.copy()

    # Limpiar las columnas antes de realizar comparaciones
    productos_df[col_productos] = productos_df[col_productos].apply(lambda x: clear_column(str(x)))
    proveedores_df[col_proveedores] = proveedores_df[col_proveedores].apply(lambda x: clear_column(str(x)))

    # Lista para almacenar los resultados
    resultados = []

    # Iterar sobre las filas del DataFrame de proveedores
    for _, proveedor in proveedores_df.iterrows():
        descripcion_proveedor = proveedor[col_proveedores]

        # Buscar coincidencias en el DataFrame de productos
        coincidencias = productos_df[
            productos_df[col_productos].str.contains(descripcion_proveedor, case=False, na=False)
        ]

        if not coincidencias.empty:
            # Si hay coincidencias, combinar las filas relevantes
            for _, producto in coincidencias.iterrows():
                resultado = {**proveedor.to_dict(), **producto.to_dict(), "match": True}
                resultados.append(resultado)
        else:
            # Si no hay coincidencias, agregar la fila del proveedor con valores nulos para los productos
            resultado = proveedor.to_dict()
            for col in productos_df.columns:
                resultado[col] = None
            resultado["match"] = False
            resultados.append(resultado)

    # Convertir la lista de resultados a un DataFrame
    return pd.DataFrame(resultados)

# Ejemplo de uso
resultado_df = obtener_coincidencias_con_match(
    productos_df=laboratorio_coincidencias_df,
    proveedores_df=copy_dfProveedores,
    col_productos="descrip1",
    col_proveedores="DESCRIPCIÓN"
)


In [8]:
resultado_df

,EAN,DESCRIPCIÓN,PVP,DROGUERIA,DTO.,CANT.X BULTO,niprod,cod_prod,cod_abrev,descrip1,...,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2,match
0,7791939000089,aspirina,"$4.904,38","$2.305,06",53%,50,11665.0,4614,4614,aspirina ginsex inf,...,E,0.0,NaN,ITGEV,0,0.0,BENITOL,NaN,NaN,True
1,7791939000089,aspirina,"$4.904,38","$2.305,06",53%,50,11666.0,7528,7528,aspirina ginsex,...,E,0.0,NaN,ITGEV,0,0.0,BENITOL,NaN,NaN,True
2,7791939000089,aspirina,"$4.904,38","$2.305,06",53%,50,11667.0,1224,1224,aspirina ginsex,...,E,0.0,NaN,ITGEV,0,0.0,BENITOL,NaN,NaN,True
3,7791939000089,aspirina,"$4.904,38","$2.305,06",53%,50,11668.0,26600,26600,aspirina ginsex expend,...,E,0.0,NaN,ITGEV,0,0.0,BENITOL,NaN,NaN,True
4,7791939000089,aspirina,"$4.904,38","$2.305,06",53%,50,11669.0,6365,6365,aspirina ginsex expend,...,E,0.0,NaN,ITGEV,0,0.0,BENITOL,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,7791939000614,enalapril,"$5.908,63","$1.654,42",72%,50,17472.0,37775,37775,fada enalapril,...,E,0.0,NaN,ITGEV,0,0.0,FADA PHARMA,NaN,NaN,True
750,7791939000614,enalapril,"$5.908,63","$1.654,42",72%,50,17473.0,29337,29337,fada enalapril,...,E,0.0,NaN,ITGEV,0,0.0,FADA PHARMA,NaN,NaN,True
751,7791939000614,enalapril,"$5.908,63","$1.654,42",72%,50,129748.0,MED-GCO02,GABU,gabuten enalapril savant pharm repetido,...,F,0.0,NaN,ITGEV,0,0.0,SAVANT,NaN,NaN,True
752,7791939000614,enalapril,"$5.908,63","$1.654,42",72%,50,127682.0,MEG-IAG-01456,MEDGCO1177,gabuten enalapril savant,...,F,0.0,NaN,ITGEV,0,0.0,SAVANT,NaN,NaN,True


In [9]:
resultado_df['atrib0']

0          BENITOL
1          BENITOL
2          BENITOL
3          BENITOL
4          BENITOL
          ...     
749    FADA PHARMA
750    FADA PHARMA
751         SAVANT
752         SAVANT
753          FABRA
Name: atrib0, Length: 754, dtype: object

In [10]:
"""
#MODULE DATAFRAME QUERY HANDLER
# Solicitar input del laboratorio
input_laboratorio = input("Ingrese el nombre del laboratorio: ")

# Filtrar los productos que coinciden parcialmente con el laboratorio ingresado
resultados = df_productos[df_productos["atrib0"].str.contains(input_laboratorio, case=False, na=False)]
resultados




# Recuento de laboratorios por nombre de laboratorio (atrib0) y filtrado para "VENT 3"
# Agrupar los datos de 'copy_dfProductos' por la columna "atrib0" (nombre del laboratorio)
# y contar la cantidad de filas por cada valor único en esta columna.
recuento_laboratorios = copy_dfProductos.groupby(copy_dfProductos["atrib0"]).size().reset_index(name='Cantidad de filas')

# Filtrar los resultados para obtener solo las filas donde el nombre del laboratorio es "VENT 3"
result = recuento_laboratorios[recuento_laboratorios["atrib0"] == "ELEA"]
result

"""

'\n#MODULE DATAFRAME QUERY HANDLER\n# Solicitar input del laboratorio\ninput_laboratorio = input("Ingrese el nombre del laboratorio: ")\n\n# Filtrar los productos que coinciden parcialmente con el laboratorio ingresado\nresultados = df_productos[df_productos["atrib0"].str.contains(input_laboratorio, case=False, na=False)]\nresultados\n\n\n\n\n# Recuento de laboratorios por nombre de laboratorio (atrib0) y filtrado para "VENT 3"\n# Agrupar los datos de \'copy_dfProductos\' por la columna "atrib0" (nombre del laboratorio)\n# y contar la cantidad de filas por cada valor único en esta columna.\nrecuento_laboratorios = copy_dfProductos.groupby(copy_dfProductos["atrib0"]).size().reset_index(name=\'Cantidad de filas\')\n\n# Filtrar los resultados para obtener solo las filas donde el nombre del laboratorio es "VENT 3"\nresult = recuento_laboratorios[recuento_laboratorios["atrib0"] == "ELEA"]\nresult\n\n'

In [11]:
# Módulo TextCleanerAndMatchFinder: 
"""
Descripción del módulo:
Este módulo realiza la limpieza y normalización de texto en las columnas de un DataFrame, específicamente en las descripciones de productos y proveedores. 
Luego, utiliza la librería RapidFuzz para encontrar coincidencias entre las descripciones de productos y proveedores basadas en su similitud.
"""

# Función de limpieza de texto para una columna específica del DataFrame
def clear_column(column):
    """
    
    Limpia y normaliza el texto en una columna para eliminar términos irrelevantes y uniformar el formato.
    
    Elimina:
    - Números, unidades y abreviaturas como 'mg', 'ml', 'g', etc.
    - Caracteres especiales como puntos, comas, guiones, etc.
    - Espacios extra y convierte todo a minúsculas.
    - Palabras comunes como 'de', 'la', 'el', 'a', etc.
    - Palabras repetidas y términos relacionados con la variabilidad en dosis/presentación.
    
    Parámetros:
        column (str): Texto a limpiar.

    Retorna:
        str: Texto limpio y normalizado.
    """
    columna_limpia = re.sub(
        r'\b(\d+|mg|comp\.?|blisters?|estuche|x|ml|unidades|marca|botella|caja|referencia|cm|g|m|f\.a\.x|'
        r'jbe\.|sol\.|env\.|lata|vial|pvo\.|sobres|gotero|cápsulas|frascos?|blister|bot\.|mcg|sp|p\.b\.)\b', 
        '', 
        column, 
        flags=re.IGNORECASE
    )
    columna_limpia = re.sub(r'[:\.\-,;()]', '', columna_limpia)
    columna_limpia = re.sub(r'\s+', ' ', columna_limpia).strip()
    columna_limpia = columna_limpia.lower()
    columna_limpia = re.sub(r'\b(?:de|la|el|en|a|y|con|para|del|sobre|por|medicamento|tratamiento|ac)\b', '', columna_limpia).strip()
    columna_limpia = re.sub(r'\b(\w+)\s+\1\b', r'\1', columna_limpia)
    columna_limpia = re.sub(r'\b(\d+mg|\d+g|\d+ml)\b', '', columna_limpia)
    return columna_limpia

# Función para cargar los datos desde un archivo CSV
def load_data(df_archivo):
    """
    Carga un archivo CSV en un DataFrame de pandas.
    
    Parámetros:
        df_archivo (str): Ruta o nombre del archivo CSV.
    
    Retorna:
        DataFrame: DataFrame con los datos cargados, o None si hubo un error.
    """
    try:
        df = pd.read_csv(df_archivo)
        return df
    except Exception as e:
        print(f"Error al cargar el archivo {df_archivo}: {e}")
        return None

# Cargar los datos desde archivos CSV
name_df_provedor = "prueba.csv"  
name_df_productos = "L_STMaestroPrd.csv"  

# Cargar los DataFrames
df_proveedor = load_data(name_df_provedor)
df_productos = load_data(name_df_productos)

# Copias de los DataFrames
copy_dfProductos = df_productos.loc[:, df_productos.notna().any()].copy()  # Eliminar columnas vacías
copy_dfProveedores = df_proveedor.copy()

# Limpiar las descripciones en ambas tablas
copy_dfProductos['descrip1_limpia'] = copy_dfProductos['descrip1'].fillna('').apply(clear_column)
copy_dfProveedores['descripcion_limpia'] = copy_dfProveedores['DESCRIPCIÓN'].fillna('').apply(clear_column)

# Crear índice para búsqueda rápida
proveedor_index = copy_dfProveedores['descripcion_limpia'].tolist()




# Función para encontrar la mejor coincidencia entre descripciones de productos y proveedores
def find_best_match(product_desc, product_row, proveedor_index, proveedor_df, threshold=80):
    """
    Encuentra la mejor coincidencia de una descripción de producto entre las descripciones de proveedores
    utilizando la librería RapidFuzz.
    
    Parámetros:
        product_desc (str): Descripción del producto.
        product_row (DataFrame row): Fila del producto con todas sus columnas.
        proveedor_index (list): Lista de descripciones limpias de proveedores.
        proveedor_df (DataFrame): DataFrame con las descripciones de proveedores.
        threshold (int): Umbral de similitud mínima para considerar una coincidencia.
    
    Retorna:
        tuple: Un diccionario con las columnas del producto, proveedor y la similitud, o None si no hay coincidencia.
    """
    best_match = process.extractOne(product_desc, proveedor_index, scorer=fuzz.ratio)
    if best_match and best_match[1] >= threshold:
        # Encontrar la fila del proveedor con la mejor coincidencia
        matched_row = proveedor_df[proveedor_df['descripcion_limpia'] == best_match[0]].iloc[0]
        
        # Combinar la fila del producto con la fila del proveedor y agregar la similitud
        match_info = product_row.to_dict()  # Convertir la fila del producto en un diccionario
        match_info.update(matched_row.to_dict())  # Actualizar con la fila del proveedor
        match_info['similaridad'] = best_match[1]  # Agregar la similitud
        
        return match_info
    return None

# Comparar las descripciones de los productos con los proveedores
matches = []
no_matches = []

# Iterar por cada producto y encontrar su mejor coincidencia
for _, prod_row in copy_dfProductos.iterrows():
    prod_desc = prod_row['descrip1_limpia']
    match_info = find_best_match(prod_desc, prod_row, proveedor_index, copy_dfProveedores)
    if match_info:
        matches.append(match_info)
    else:
        no_matches.append(prod_desc)

# Convertir los resultados en DataFrames
df_matches = pd.DataFrame(matches)
df_no_matches = pd.DataFrame(no_matches, columns=['producto_sin_match'])

# Reordenar columnas para que la columna 'similaridad' esté al principio
cols = ['similaridad'] + [col for col in df_matches.columns if col != 'similaridad']
df_matches = df_matches[cols]

# Mostrar los resultados
print(f"Coincidencias encontradas: {len(df_matches)}")
print(df_matches)

print(f"\nProductos sin coincidencias: {len(df_no_matches)}")
print(df_no_matches)

C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_14060\1070585160.py:54: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_archivo)


Coincidencias encontradas: 105
     similaridad    niprod cod_prod cod_abrev  \
0     100.000000   98611.0    31128     31128   
1     100.000000   14745.0    11631     11631   
2     100.000000   14746.0    11630     11630   
3      84.210526  113090.0     1177      1177   
4      84.210526  113091.0    51966     51966   
..           ...       ...      ...       ...   
100    91.228070   91647.0     6839      6839   
101    91.228070   96762.0    10069     10069   
102    82.352941  107037.0    27370     27370   
103    82.352941  107038.0    30569     30569   
104    82.352941  105725.0    45577     45577   

                                           descrip1  \
0                    ASPIRINA 4 A 500 mg comp.x 100   
1                      ASPIRINA G 500 mg comp.x 100   
2                       ASPIRINA G 500 mg comp.x 20   
3                            ASPIRINETAS comp.x 100   
4                             ASPIRINETAS comp.x 28   
..                                              ..

In [1]:
import pandas as pd
import re
from rapidfuzz import fuzz, process

# Función de limpieza de texto para una columna específica del DataFrame
def clear_column(column):
    """
    Limpia y normaliza el texto en una columna para eliminar términos irrelevantes y uniformar el formato.
    """
    # Eliminamos palabras irrelevantes como unidades, envases, etc.
    columna_limpia = re.sub(
        r'\b(\d+|mg|comp\.?|blisters?|estuche|x|ml|unidades|marca|botella|caja|referencia|cm|g|m|f\.a\.x|'
        r'jbe\.|sol\.|env\.|lata|vial|pvo\.|sobres|gotero|cápsulas|frascos?|blister|bot\.|mcg|sp|p\.b\.)\b', 
        '', 
        column, 
        flags=re.IGNORECASE
    )
    # Reemplazar saltos de línea por espacios
    columna_limpia = re.sub(r'\n', ' ', columna_limpia)
    columna_limpia = re.sub(r'[:\.\-,;()]', '', columna_limpia)
    columna_limpia = re.sub(r'\s+', ' ', columna_limpia).strip()
    columna_limpia = columna_limpia.lower()
    # Eliminamos palabras comunes (como 'de', 'la', etc.) que no ayudan a la comparación
    columna_limpia = re.sub(r'\b(?:de|la|el|en|a|y|con|para|del|sobre|por|medicamento|tratamiento|ac|se)\b', '', columna_limpia).strip()
    columna_limpia = re.sub(r'\b(\w+)\s+\1\b', r'\1', columna_limpia)
    # Aseguramos que unidades como mg, ml, etc. no interfieran
    columna_limpia = re.sub(r'\b(\d+mg|\d+g|\d+ml)\b', '', columna_limpia)
    return columna_limpia

# Función para cargar los datos desde un archivo CSV
def load_data(df_archivo):
    """
    Carga un archivo CSV en un DataFrame de pandas.
    """
    try:
        df = pd.read_csv(df_archivo)
        return df
    except Exception as e:
        print(f"Error al cargar el archivo {df_archivo}: {e}")
        return None

# Función para filtrar productos por laboratorio
def laboratorio_input_df(df, columna):
    """
    Busca coincidencias parciales en una columna de un DataFrame basado en una entrada del usuario.
    """
    input_name = input("Ingrese el nombre del laboratorio: ").strip()
    tablas_coincidencias = df[df[columna].str.contains(input_name, case=False, na=False)]

    if not tablas_coincidencias.empty:
        print(f"Se encontraron {len(tablas_coincidencias)} productos coincidentes del laboratorio '{input_name}'.")
        return tablas_coincidencias
    else:
        print(f"No se encontraron productos del laboratorio '{input_name}'.")
        return None

# Función para normalizar celdas con múltiples productos
def normalize_dataframe(df, column_to_split):
    """
    Normaliza un DataFrame separando celdas con múltiples productos en filas individuales.
    Args:
        df (pd.DataFrame): DataFrame original.
        column_to_split (str): Nombre de la columna a normalizar.
    Returns:
        pd.DataFrame: DataFrame normalizado con un producto por fila.
    """
    # Dividir las celdas problemáticas en listas usando \n como delimitador
    df[column_to_split] = df[column_to_split].astype(str).str.split(r'\n')

    # Expandir las listas en múltiples filas
    df_normalized = df.explode(column_to_split, ignore_index=True)

    # Limpiar espacios extra en las celdas
    df_normalized[column_to_split] = df_normalized[column_to_split].str.strip()

    return df_normalized

# Función de comparación difusa
def comparar_similitud(texto1, texto2, umbral=80):
    """
    Compara dos textos utilizando similitud difusa (fuzzy matching) con un umbral configurable.
    """
    # Limpiar ambos textos antes de la comparación
    texto1_limpio = clear_column(texto1)
    texto2_limpio = clear_column(texto2)
    
    # Calcular la similitud entre los dos textos usando partial_ratio
    similitud = fuzz.partial_ratio(texto1_limpio, texto2_limpio)
    
    # Verificar si la similitud es mayor o igual al umbral
    return similitud >= umbral, similitud

# Cargar los datos desde archivos CSV
name_df_provedor = "prueba2.csv"  
name_df_productos = "L_STMaestroPrd.csv"  

df_proveedor = load_data(name_df_provedor)
df_productos = load_data(name_df_productos)

# Normalizar las columnas con múltiples productos
df_proveedor = normalize_dataframe(df_proveedor, 'DESCRIPCIÓN')
df_productos = normalize_dataframe(df_productos, 'descrip1')

# Copias de los DataFrames
copy_dfProductos = df_productos.loc[:, df_productos.notna().any()].copy()  # Eliminar columnas vacías
copy_dfProveedores = df_proveedor.copy()

# Limpiar las descripciones en ambas tablas
copy_dfProductos['descrip1_limpia'] = copy_dfProductos['descrip1'].fillna('').apply(clear_column)
copy_dfProveedores['descripcion_limpia'] = copy_dfProveedores['DESCRIPCIÓN'].fillna('').apply(clear_column)

# Filtrar los productos por laboratorio
columna_nombre = 'atrib0'  # Cambia por el nombre de la columna que contiene el laboratorio
laboratorio_coincidencias_df = laboratorio_input_df(copy_dfProductos, columna_nombre)

if laboratorio_coincidencias_df is not None:
    # Crear índice para búsqueda rápida
    proveedor_index = copy_dfProveedores['descripcion_limpia'].tolist()

    matches = []

    # Recorrer productos del proveedor y buscar todas las coincidencias con los productos del laboratorio
    for _, prov_row in copy_dfProveedores.iterrows():
        prov_desc = prov_row['descripcion_limpia']

        # Buscar todas las coincidencias que superen el umbral de similitud
        all_matches = process.extract(prov_desc, laboratorio_coincidencias_df['descrip1_limpia'], scorer=fuzz.partial_ratio, limit=None)
        valid_matches = [match for match in all_matches if match[1] >= 80]  # Filtrar por umbral de 80

        if valid_matches:
            for match in valid_matches:
                matched_row = laboratorio_coincidencias_df[
                    laboratorio_coincidencias_df['descrip1_limpia'] == match[0]
                ].iloc[0]  # Obtener la fila coincidente del laboratorio

                # Combinar la información del producto del proveedor y del laboratorio
                match_info = prov_row.to_dict()  # Información del proveedor
                match_info.update(matched_row.to_dict())  # Información del producto del laboratorio
                match_info['similaridad'] = match[1]  # Similaridad

                matches.append(match_info)
        else:
            # Si no hay coincidencia, incluir sólo la información del proveedor
            match_info = prov_row.to_dict()
            match_info['similaridad'] = 0  # Similaridad = 0

            # Agregar columnas vacías para los datos del laboratorio
            for col in laboratorio_coincidencias_df.columns:
                if col not in match_info:
                    match_info[col] = None

            matches.append(match_info)

    # Convertir la lista de matches a DataFrame
    df_matches = pd.DataFrame(matches)

    # Reordenar columnas para que 'similaridad' esté al principio
    cols = ['similaridad'] + [col for col in df_matches.columns if col != 'similaridad']
    df_matches = df_matches[cols]

    # Filtrar las coincidencias que no cumplen con el umbral de similitud
    df_matches_bajo_umbral = df_matches[df_matches['similaridad'] < 80]
    print(f"\nCoincidencias con similitud menor al umbral de 80:")
    print(df_matches_bajo_umbral)

    # Mostrar resultados
    print(f"\nCoincidencias encontradas: {len(df_matches[df_matches['similaridad'] > 0])}")
    print(df_matches)

    # Productos del laboratorio sin coincidencias
    productos_sin_coincidencia = laboratorio_coincidencias_df[~laboratorio_coincidencias_df['descrip1_limpia'].isin(
        df_matches[df_matches['similaridad'] > 0]['descrip1_limpia']
    )]

    print(f"\nProductos del laboratorio sin coincidencias: {len(productos_sin_coincidencia)}")
    print(productos_sin_coincidencia)
else:
    print("No se encontraron coincidencias para el laboratorio especificado.")


C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_6984\3094562052.py:36: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_archivo)


No se encontraron productos del laboratorio 'VEN3'.
No se encontraron coincidencias para el laboratorio especificado.
